各JSONファイルに対して、フルテキストのみを抽出し、ファイル名、フルテキストのdataframeを作成
フルテキストをGPT、Geminiに投げて、構造化した組み入れ・除外基準を返してもらう

まずはpdf extraction.ipynbで、1つのフォルダにまとめたpdfから情報を抜き出してjsonファイルを作る。ADobeのAPIを使用する

In [ ]:
pip install pdfservices-sdk

In [ ]:
import zipfile
import os

# ZIPファイルのパス
zip_path = "your_zip_file_path.zip"  # ここに実際のZIPファイルのパスを指定してください

# パスが本当にZIPファイルなのか確認
if os.path.isdir(zip_path):
    print("これはディレクトリです。ZIPファイルではありません。")
    # ディレクトリ内のファイルを確認
    files = os.listdir(zip_path)
    print(f"ディレクトリ内のファイル: {files[:10]}")  # 最初の10個だけ表示
elif os.path.isfile(zip_path):
    print("これはファイルです。")
    # おそらく拡張子が抜けている可能性があります
    if not zip_path.endswith('.zip'):
        print("ZIPファイルの拡張子がありません。")
        zip_path = zip_path + '.zip'
        print(f"拡張子を追加したパス: {zip_path}")
    
    # ZIPファイルの中身を確認
    try:
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            # ZIPファイル内のファイル一覧を取得
            file_list = zip_ref.namelist()
            print(f"ZIP内のファイル数: {len(file_list)}")
            print(f"最初の10個のファイル: {file_list[:10]}")
    except zipfile.BadZipFile:
        print("無効なZIPファイルです。")
    except FileNotFoundError:
        print("ファイルが見つかりません。")
else:
    print("指定されたパスが存在しません。")

In [ ]:
import zipfile
import os
import shutil
from pathlib import Path

# パスの設定
zip_path = "your_zip_file_path.zip"  # ここに実際のZIPファイルのパスを指定してください
target_dir = "extracted_pdfs"  # 抽出先のディレクトリ名

# 対象ディレクトリが存在しない場合は作成
if not os.path.exists(target_dir):
    os.makedirs(target_dir)
    print(f"ディレクトリを作成しました: {target_dir}")
else:
    print(f"ディレクトリはすでに存在します: {target_dir}")

# ZIPファイルを開いてPDFファイルを抽出
try:
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        # ZIPファイル内のファイル一覧を取得
        file_list = [f for f in zip_ref.namelist() if f.lower().endswith('.pdf')]
        
        print(f"PDFファイル数: {len(file_list)}")
        
        # 進捗表示のためのカウンタ
        counter = 0
        
        # PDFファイルを抽出
        for file_name in file_list:
            try:
                # ファイル名のみを取得 (パスを除去)
                base_name = os.path.basename(file_name)
                
                # ファイル名を「-」の前の部分だけに短縮する
                name_parts = os.path.splitext(base_name)  # 拡張子を分離
                file_name_without_ext = name_parts[0]
                extension = name_parts[1]  # 拡張子 (.pdf)
                
                # ハイフンで分割して最初の部分だけを取得
                if '-' in file_name_without_ext:
                    shortened_name = file_name_without_ext.split('-')[0].strip()
                    base_name = shortened_name + extension
                else:
                    # ハイフンがない場合はそのまま使用
                    base_name = file_name_without_ext + extension
                
                # 対象パス
                target_path = os.path.join(target_dir, base_name)
                
                # ファイルを抽出して保存
                with zip_ref.open(file_name) as source, open(target_path, 'wb') as target:
                    shutil.copyfileobj(source, target)
                
                counter += 1
                if counter % 50 == 0:  # 50ファイルごとに進捗を表示
                    print(f"進捗: {counter}/{len(file_list)} ファイルを処理しました")
                    
            except Exception as e:
                print(f"ファイル {file_name} の処理中にエラーが発生しました: {e}")
        
        print(f"完了しました。{counter}/{len(file_list)} ファイルを抽出しました。")

except Exception as e:
    print(f"エラーが発生しました: {e}")

In [ ]:
import os

# フォルダのパス
folder_path = "extracted_pdfs"  # 抽出したPDFファイルが保存されているフォルダのパス

# フォルダが存在するか確認
if not os.path.exists(folder_path):
    print(f"エラー: フォルダが存在しません: {folder_path}")
else:
    # フォルダ内のファイルをリストアップ
    files = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]
    
    # ファイル数を表示
    print(f"フォルダ内のファイル総数: {len(files)}")
    
    # PDFファイルの数を確認
    pdf_files = [f for f in files if f.lower().endswith('.pdf')]
    print(f"PDFファイルの数: {len(pdf_files)}")
    
    # 最初の10個のファイル名を表示
    if files:
        print("\n最初の10個のファイル名:")
        for i, file in enumerate(files[:10]):
            print(f"{i+1}. {file}")


In [ ]:
import glob
import os

# 絶対パスを使用
folder_path = "extracted_pdfs"  # 抽出したPDFファイルが保存されているフォルダのパス
pdf_files = glob.glob(os.path.join(folder_path, "*.pdf"))

print(f"PDFファイルの数: {len(pdf_files)}")

In [ ]:
import os

# 現在の作業ディレクトリを取得して表示
current_directory = os.getcwd()
print(f"現在の作業ディレクトリ: {current_directory}")

In [ ]:
import os

# 作業ディレクトリを設定
target_directory = "extracted_pdfs"  # 抽出したPDFファイルが保存されているフォルダのパス
os.chdir(target_directory)

# 設定後の作業ディレクトリを確認
current_directory = os.getcwd()
print(f"現在の作業ディレクトリ: {current_directory}")

In [ ]:
import os
import logging
import zipfile
from datetime import datetime

from adobe.pdfservices.operation.auth.service_principal_credentials import ServicePrincipalCredentials
from adobe.pdfservices.operation.exception.exceptions import ServiceApiException, ServiceUsageException, SdkException
from adobe.pdfservices.operation.io.cloud_asset import CloudAsset
from adobe.pdfservices.operation.io.stream_asset import StreamAsset
from adobe.pdfservices.operation.pdf_services import PDFServices
from adobe.pdfservices.operation.pdf_services_media_type import PDFServicesMediaType
from adobe.pdfservices.operation.pdfjobs.jobs.extract_pdf_job import ExtractPDFJob
from adobe.pdfservices.operation.pdfjobs.params.extract_pdf.extract_element_type import ExtractElementType
from adobe.pdfservices.operation.pdfjobs.params.extract_pdf.extract_renditions_element_type import ExtractRenditionsElementType
from adobe.pdfservices.operation.pdfjobs.params.extract_pdf.extract_pdf_params import ExtractPDFParams
from adobe.pdfservices.operation.pdfjobs.result.extract_pdf_result import ExtractPDFResult

class ExtractTextTableInfoWithFiguresTablesRenditionsFromPDF:
    def __init__(self, pdf_file):
        try:
            file = open(pdf_file, 'rb')
            input_stream = file.read()
            file.close()

            # Initial setup, create credentials instance
            credentials = ServicePrincipalCredentials(
                client_id="your_client_id",
                client_secret="your_client_secret"  # Replace with your actual client secret
            )

            # Creates a PDF Services instance
            pdf_services = PDFServices(credentials=credentials)

            # Creates an asset(s) from source file(s) and upload
            input_asset = pdf_services.upload(input_stream=input_stream, mime_type=PDFServicesMediaType.PDF)

            # Create parameters for the job
            extract_pdf_params = ExtractPDFParams(
                elements_to_extract=[ExtractElementType.TEXT, ExtractElementType.TABLES],
                elements_to_extract_renditions=[ExtractRenditionsElementType.TABLES, ExtractRenditionsElementType.FIGURES],
            )

            # Creates a new job instance
            extract_pdf_job = ExtractPDFJob(input_asset=input_asset, extract_pdf_params=extract_pdf_params)

            # Submit the job and gets the job result
            location = pdf_services.submit(extract_pdf_job)
            pdf_services_response = pdf_services.get_job_result(location, ExtractPDFResult)

            # Get content from the resulting asset(s)
            result_asset: CloudAsset = pdf_services_response.get_result().get_resource()
            stream_asset: StreamAsset = pdf_services.get_content(result_asset)

            # Creates an output stream and copy stream asset's content to it
            output_file_path = "your_output_file_path.zip"  # Specify your output file path here
            with open(output_file_path, "wb") as file:
                file.write(stream_asset.get_input_stream())

            # ---- ここからフォルダへの解凍処理を追加 ----
            # PDF ファイルと同じフォルダ名(拡張子除く)を作成して、そこに解凍する
            folder_name = os.path.splitext(os.path.basename(pdf_file))[0]  # PDF と同名のフォルダ名
            output_folder_path = os.path.join(os.path.dirname(pdf_file), folder_name)

            # 同名のフォルダが存在していない場合のみ解凍を行う
            if os.path.exists(output_folder_path):
                logging.info(f"フォルダ '{output_folder_path}' は既に存在しています。解凍をスキップします。")
            else:
                os.makedirs(output_folder_path, exist_ok=True)
                with zipfile.ZipFile(output_file_path, 'r') as zip_ref:
                    zip_ref.extractall(output_folder_path)
                logging.info(f"zipファイルを解凍して '{output_folder_path}' に保存しました。")
            # ---- ここまで追加 ----

        except (ServiceApiException, ServiceUsageException, SdkException) as e:
            logging.exception(f'Exception encountered while executing operation: {e}')

In [ ]:
from tqdm import tqdm

if __name__ == "__main__":
    for pdf_file in tqdm(pdf_files, desc="Processing PDFs"):
        ExtractTextTableInfoWithFiguresTablesRenditionsFromPDF(pdf_file)

In [ ]:
# PDFファイル一覧を取得
pdf_files = glob.glob(os.path.join(folder_path, "*.pdf"))
print(f"合計 {len(pdf_files)} 個のPDFファイルを検出しました。")

In [ ]:
import os
import glob
import logging

# ログ設定
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# フォルダのパス
folder_path = "extracted_pdfs"  # 抽出したPDFファイルが保存されているフォルダのパス

# PDFファイル一覧を取得
pdf_files = glob.glob(os.path.join(folder_path, "*.pdf"))
logging.info(f"合計 {len(pdf_files)} 個のPDFファイルを検出しました。")

# 既に処理済みのファイルを確認（同名のフォルダが存在するか）
processed_files = []
remaining_files = []

for pdf_file in pdf_files:
    folder_name = os.path.splitext(os.path.basename(pdf_file))[0]
    output_folder_path = os.path.join(os.path.dirname(pdf_file), folder_name)
    
    if os.path.exists(output_folder_path):
        processed_files.append(pdf_file)
    else:
        remaining_files.append(pdf_file)

print(f"処理済み: {len(processed_files)} ファイル")
print(f"未処理: {len(remaining_files)} ファイル")

# 詳細情報を表示（オプション）
print("\n処理済みファイルの例（最初の5つ）:")
for i, file in enumerate(processed_files[:5]):
    print(f"{i+1}. {os.path.basename(file)}")

print("\n未処理ファイルの例（最初の5つ）:")
for i, file in enumerate(remaining_files[:5]):
    print(f"{i+1}. {os.path.basename(file)}")

In [ ]:
#API key 変えて再挑戦

class ExtractTextTableInfoWithFiguresTablesRenditionsFromPDF:
    def __init__(self, pdf_file):
        try:
            file = open(pdf_file, 'rb')
            input_stream = file.read()
            file.close()

            # Initial setup, create credentials instance
            credentials = ServicePrincipalCredentials(
                client_id='your_client_id',  # Replace with your actual client ID
                client_secret=  'your_client_secret'  # Replace with your actual client secret
            )

            # Creates a PDF Services instance
            pdf_services = PDFServices(credentials=credentials)

            # Creates an asset(s) from source file(s) and upload
            input_asset = pdf_services.upload(input_stream=input_stream, mime_type=PDFServicesMediaType.PDF)

            # Create parameters for the job
            extract_pdf_params = ExtractPDFParams(
                elements_to_extract=[ExtractElementType.TEXT, ExtractElementType.TABLES],
                elements_to_extract_renditions=[ExtractRenditionsElementType.TABLES, ExtractRenditionsElementType.FIGURES],
            )

            # Creates a new job instance
            extract_pdf_job = ExtractPDFJob(input_asset=input_asset, extract_pdf_params=extract_pdf_params)

            # Submit the job and gets the job result
            location = pdf_services.submit(extract_pdf_job)
            pdf_services_response = pdf_services.get_job_result(location, ExtractPDFResult)

            # Get content from the resulting asset(s)
            result_asset: CloudAsset = pdf_services_response.get_result().get_resource()
            stream_asset: StreamAsset = pdf_services.get_content(result_asset)

            # Creates an output stream and copy stream asset's content to it
            output_file_path = 'extractTextTableInfoWithFiguresTablesRenditionsFromPDF.zip'
            with open(output_file_path, "wb") as file:
                file.write(stream_asset.get_input_stream())

            # ---- ここからフォルダへの解凍処理を追加 ----
            # PDF ファイルと同じフォルダ名(拡張子除く)を作成して、そこに解凍する
            folder_name = os.path.splitext(os.path.basename(pdf_file))[0]  # PDF と同名のフォルダ名
            output_folder_path = os.path.join(os.path.dirname(pdf_file), folder_name)

            # 同名のフォルダが存在していない場合のみ解凍を行う
            if os.path.exists(output_folder_path):
                logging.info(f"フォルダ '{output_folder_path}' は既に存在しています。解凍をスキップします。")
            else:
                os.makedirs(output_folder_path, exist_ok=True)
                with zipfile.ZipFile(output_file_path, 'r') as zip_ref:
                    zip_ref.extractall(output_folder_path)
                logging.info(f"zipファイルを解凍して '{output_folder_path}' に保存しました。")
            # ---- ここまで追加 ----

        except (ServiceApiException, ServiceUsageException, SdkException) as e:
            logging.exception(f'Exception encountered while executing operation: {e}')

In [ ]:
#remaining_filesに適用
from tqdm import tqdm

if __name__ == "__main__":
    for pdf_file in tqdm(remaining_files, desc="Processing PDFs"):
        ExtractTextTableInfoWithFiguresTablesRenditionsFromPDF(pdf_file)

In [ ]:
import glob
jsons = glob.glob("fulltext/*/*.json")
print(len(jsons))

In [ ]:
# jsonファイルを読み込む
import json
with open(jsons[0]) as f:
    data = json.load(f)

In [ ]:
import os
import json
import pandas as pd
from typing import List, Dict, Any, Optional


def clean_text(text: str) -> str:
    """
    Remove extra whitespace and normalize spacing in the given text.
    """
    return ' '.join(text.split())


def should_start_new_segment(element: Dict[str, Any]) -> bool:
    """
    Determine whether a new segment should start based on element attributes.
    """
    path = element.get('Path', '')
    text = element.get('Text', '')

    return (
        path.endswith(('H1', 'Title')) or
        'ABSTRACT' in text
    )


def extract_text_segments(document_data: Dict[str, Any]) -> List[str]:
    """
    Extract and segment text content from PDF data while removing formatting information.
    """
    segments: List[str] = []
    current_segment: List[str] = []

    elements = document_data.get('elements', [])

    for element in elements:
        # Skip non-text elements or empty text
        if 'Text' not in element or not element['Text'].strip():
            continue

        # Skip figure captions and other non-content elements
        if element.get('Path', '').endswith(('Figure', 'Lbl')):
            continue

        text = clean_text(element['Text'])

        # Start new segment if needed
        if should_start_new_segment(element):
            if current_segment:
                segments.append(' '.join(current_segment))
                current_segment = []
            current_segment.append(text)
        else:
            current_segment.append(text)

    if current_segment:
        segments.append(' '.join(current_segment))

    return segments


def process_pdf_content(document_content: str) -> List[str]:
    """
    Process PDF content (JSON formatted string) and return segmented text.
    """
    try:
        pdf_data = json.loads(document_content)
        segments = extract_text_segments(pdf_data)
        return segments
    except json.JSONDecodeError:
        return ["Error: Invalid JSON data"]
    except Exception as e:
        return [f"Error processing document: {str(e)}"]


def read_json_file(file_path: str) -> Optional[str]:
    """
    Safely read JSON file content.
    """
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            content = f.read()
        return content
    except Exception as e:
        print(f"Error reading file '{file_path}': {str(e)}")
        return None


def extract_folder_name(file_path: str) -> str:
    """
    Extract folder name from file path, specifically the one after 'fulltext'.
    """
    normalized_path = os.path.normpath(file_path)
    path_parts = normalized_path.split(os.sep)
    try:
        fulltext_index = path_parts.index('fulltext')
        if len(path_parts) > fulltext_index + 1:
            return path_parts[fulltext_index + 1]
    except ValueError:
        pass

    return "Unknown"


def create_segments_dataframe(json_paths: List[str]) -> pd.DataFrame:
    """
    Create a DataFrame such that each folder corresponds to one row,
    and each row stores a list of segments in the 'segments' column.
    """
    all_data = []
    for json_path in json_paths:
        folder_name = extract_folder_name(json_path)
        content = read_json_file(json_path)

        # フォルダ単位で segments をまとめる
        segments = []
        if content:
            segments = process_pdf_content(content)

        all_data.append({
            'folder_name': folder_name,
            # まとめた segment をひとつのリストとして保存
            'segments': segments
        })

    return pd.DataFrame(all_data)


def save_segments_to_csv(
    json_paths: List[str],
    output_path: str = 'full_text_without_tab_and_fig_output.csv'
) -> pd.DataFrame:
    """
    Process JSON files and save segments to CSV.
    """
    df = create_segments_dataframe(json_paths)
    df.to_csv(output_path, index=False, encoding='utf-8')
    print(f"Saved segments to {output_path}")
    return df




In [ ]:
# Usage example (will only run when this file is executed directly).
if __name__ == "__main__":
    df = save_segments_to_csv(jsons)
    print(f"Created DataFrame with {len(df)} rows")
    print("\nFirst few rows:")
    print(df.head())


これでフルテキストを抽出したcsvができた。以下はjudgementのipynbに続く

In [ ]:
type(df["segments"][0])

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("full_text_without_tab_and_fig_output.csv", encoding='utf-8')

In [ ]:
df.head()

df["segments"] = df["segments"].apply(str)

type(df["segments"][0])

In [ ]:
def clean_text(text):
    """
    Clean text by removing unnecessary newlines and normalizing spaces.

    Parameters:
    text (str): Input text with unwanted newlines

    Returns:
    str: Cleaned text with proper spacing
    """
    # Remove single newline characters
    text = text.replace('\n', '')

    # Normalize spaces (remove multiple spaces)
    text = ' '.join(text.split())

    return text

In [ ]:
df['segments'] = df['segments'].apply(clean_text)

In [ ]:
display(df)

In [ ]:
pip install anthropic

In [ ]:
import time
import json
import pandas as pd
from typing import Optional, List
from dataclasses import dataclass
from pydantic import BaseModel
from openai import OpenAI  # Anthropic の代わりに OpenAI SDK を使用

@dataclass
class FullTextScreeningResult:
    judgement: Optional[bool]  # 含める(True)/含めない(False)
    reason: Optional[str]       # 理由
    error: Optional[str] = None # APIエラーなどがあった場合


class FullTextDecision(BaseModel):
    judgement: bool
    reason: str


def get_fulltext_completion(
    system_prompt: str,
    prompt: str,
    client: OpenAI,  # OpenAI クライアントを使用
    model: str = "anthropic/claude-3-7-sonnet",  # OpenRouter のモデル名
    temperature: float = 0.0
) -> FullTextDecision:
    """
    OpenRouter経由でClaudeにフルテキストを読ませ、含める／含めないの判断と理由を返してもらう関数。
    """
    # システムプロンプトでフォーマットの指示を行う
    enhanced_system_prompt = f"""
    {system_prompt}
    
    以下のテキストを評価し、指定された基準に基づいて含めるか除外するかを判断してください。
    結果は以下のXMLフォーマットで必ず返してください:

    <judgement>true または false</judgement>
    <reason>判断の詳細な理由</reason>
    """

    # OpenRouter APIを呼び出し (OpenAI の形式を使用)
    response = client.chat.completions.create(
        model=model,
        temperature=temperature,
        messages=[
            {"role": "system", "content": enhanced_system_prompt},
            {"role": "user", "content": prompt}
        ],
        extra_headers={
            "HTTP-Referer": "https://yourwebsite.com",  # 必要に応じて変更
            "X-Title": "SystematicReviewApp",          # 必要に応じて変更
        }
    )

    # レスポンスからテキストを抽出
    response_content = response.choices[0].message.content

    # XMLタグから判断と理由を抽出
    try:
        # 判断を抽出
        judgement_start = response_content.find("<judgement>") + len("<judgement>")
        judgement_end = response_content.find("</judgement>")
        judgement_text = response_content[judgement_start:judgement_end].strip().lower()
        judgement = judgement_text == "true"

        # 理由を抽出
        reason_start = response_content.find("<reason>") + len("<reason>")
        reason_end = response_content.find("</reason>")
        reason = response_content[reason_start:reason_end].strip()

        return FullTextDecision(judgement=judgement, reason=reason)
    except Exception as e:
        # XMLタグがない場合は、テキスト全体を理由として扱い、判断をFalseとする
        print(f"Error parsing XML response: {e}")
        print(f"Raw response: {response_content}")

        # シンプルなヒューリスティックで判断を試みる
        judgement = "含める" in response_content.lower() or "include" in response_content.lower()

        return FullTextDecision(
            judgement=judgement,
            reason="APIレスポンスからの抽出に失敗しました。テキスト全体: " + response_content[:200] + "..."
        )


def process_fulltext_abstracts(
    df: pd.DataFrame,
    system_prompt: str,
    client: OpenAI,  # OpenAI クライアントを使用
    text_column: str = "segments",  # フルテキストが入っている列名
    max_retries: int = 3,
    retry_delay: int = 5,
    model: str = "anthropic/claude-3-7-sonnet"  # OpenRouter のモデル名
) -> List[FullTextScreeningResult]:
    """
    DataFrameの各行に含まれるフルテキストをClaudeに投げて、
    含める／含めない＆理由をまとめて返す。
    """
    results = []

    for i, row in df.iterrows():
        text_to_screen = row.get(text_column, "")
        current_result = None
        error_message = None

        for attempt in range(max_retries):
            try:
                # Claudeに問い合わせ
                decision = get_fulltext_completion(
                    system_prompt=system_prompt,
                    prompt=text_to_screen,
                    client=client,
                    model=model
                )
                current_result = FullTextScreeningResult(
                    judgement=decision.judgement,
                    reason=decision.reason
                )
                break  # 成功したらリトライループを抜ける

            except Exception as e:
                error_message = str(e)
                print(f"[{i}] Error (attempt {attempt+1}/{max_retries}): {error_message}")
                if attempt < max_retries - 1:
                    print(f"Retrying in {retry_delay} seconds...")
                    time.sleep(retry_delay)
                else:
                    print("Max retries reached. Moving to next.")

        # current_resultに成功結果が入っていなければ、エラーとして記録
        if not current_result:
            current_result = FullTextScreeningResult(
                judgement=None,
                reason=None,
                error=error_message
            )

        results.append(current_result)

    return results


def process_fulltext_screening(
    df: pd.DataFrame,
    system_prompt: str,
    client: OpenAI,  # OpenAI クライアントを使用
    text_column: str = "segments",
    model: str = "anthropic/claude-3-7-sonnet"  # OpenRouter のモデル名
) -> pd.DataFrame:
    """
    フルテキストスクリーニングを実行し、結果を元のDataFrameに追加して返す

    Args:
        df: 入力DataFrame
        system_prompt: システムプロンプト
        client: OpenAI APIクライアント
        text_column: テキストが含まれる列名
        model: 使用するモデル名

    Returns:
        結果を追加したDataFrame
    """
    # 元のdfを変更しないようにコピー
    df = df.copy()

    # 各テキストに対してスクリーニングを実行
    results = process_fulltext_abstracts(
        df=df,
        system_prompt=system_prompt,
        client=client,
        text_column=text_column,
        model=model
    )

    # 結果をDataFrameに追加
    df['claude_decision'] = [r.judgement for r in results]
    df['claude_reason'] = [r.reason for r in results]
    df['claude_error'] = [r.error for r in results]

    return df

In [ ]:
# テキストクリーニング関数
def clean_text(text):
    if isinstance(text, str):
        # リスト記法が文字列に含まれている場合、それを除去
        if text.startswith('[') and ']' in text:
            text = text.replace('[', '', 1)
            text = text.replace(']', '', 1)
        
        # 引用符を除去
        text = text.replace('"', '').replace("'", "")
        
        # 特殊文字をチェック
        import re
        text = re.sub(r'[^\x00-\x7F]+', ' ', text)  # 非ASCII文字を空白に置換
        
        return text.strip()
    elif isinstance(text, list):
        # リストの場合は各要素を文字列に変換して結合
        return ' '.join([clean_text(str(item)) for item in text])
    else:
        # その他の型の場合は文字列に変換
        return str(text)

# データフレーム全体のテキスト列をクリーニング
text_column = "segments"  # あなたのテキスト列名
df_cleaned = df.copy()
df_cleaned[text_column] = df_cleaned[text_column].apply(clean_text)

In [ ]:
import time
import json
import pandas as pd
import datetime
from typing import Optional, List
from dataclasses import dataclass
from pydantic import BaseModel
from openai import OpenAI  # OpenAI SDK を使用

@dataclass
class FullTextScreeningResult:
    judgement: Optional[bool]  # 含める(True)/含めない(False)
    reason: Optional[str]       # 理由
    error: Optional[str] = None # APIエラーなどがあった場合


class FullTextDecision(BaseModel):
    judgement: bool
    reason: str


# テキストクリーニング関数
def clean_text(text):
    if isinstance(text, str):
        # リスト記法が文字列に含まれている場合、それを除去
        if text.startswith('[') and ']' in text:
            text = text.replace('[', '', 1)
            text = text.replace(']', '', 1)
        
        # 引用符を除去
        text = text.replace('"', '').replace("'", "")
        
        # 特殊文字をチェック
        import re
        text = re.sub(r'[^\x00-\x7F]+', ' ', text)  # 非ASCII文字を空白に置換
        
        return text.strip()
    elif isinstance(text, list):
        # リストの場合は各要素を文字列に変換して結合
        return ' '.join([clean_text(str(item)) for item in text])
    else:
        # その他の型の場合は文字列に変換
        return str(text)


def get_fulltext_completion(
    system_prompt: str,
    prompt: str,
    client: OpenAI,  # OpenAI クライアントを使用
    model: str = "anthropic/claude-3-7-sonnet",  # OpenRouter のモデル名
    temperature: float = 0.0,
    timeout: int = 60  # タイムアウト設定（秒）
) -> FullTextDecision:
    """
    OpenRouter経由でClaudeにフルテキストを読ませ、含める／含めないの判断と理由を返してもらう関数。
    """
    # クリーニング処理を追加
    prompt = clean_text(prompt)
    
    # システムプロンプトでフォーマットの指示を行う
    enhanced_system_prompt = f"""
    {system_prompt}
    
    以下のテキストを評価し、指定された基準に基づいて含めるか除外するかを判断してください。
    結果は以下のXMLフォーマットで必ず返してください:

    <judgement>true または false</judgement>
    <reason>判断の詳細な理由</reason>
    """

    # OpenRouter APIを呼び出し (OpenAI の形式を使用)
    try:
        response = client.chat.completions.create(
            model=model,
            temperature=temperature,
            messages=[
                {"role": "system", "content": enhanced_system_prompt},
                {"role": "user", "content": prompt}
            ],
            extra_headers={
                "HTTP-Referer": "https://yourwebsite.com",  # 必要に応じて変更
                "X-Title": "SystematicReviewApp",          # 必要に応じて変更
            },
            timeout=timeout  # タイムアウト設定
        )

        # レスポンスからテキストを抽出
        if hasattr(response, 'choices') and len(response.choices) > 0:
            response_content = response.choices[0].message.content
        else:
            raise ValueError(f"Unexpected response format: {response}")

        # XMLタグから判断と理由を抽出
        judgement_start = response_content.find("<judgement>") + len("<judgement>")
        judgement_end = response_content.find("</judgement>")
        judgement_text = response_content[judgement_start:judgement_end].strip().lower()
        judgement = judgement_text == "true"

        # 理由を抽出
        reason_start = response_content.find("<reason>") + len("<reason>")
        reason_end = response_content.find("</reason>")
        reason = response_content[reason_start:reason_end].strip()

        return FullTextDecision(judgement=judgement, reason=reason)
    except Exception as e:
        # エラーが発生した場合はデバッグ情報を出力
        print(f"Error in API request: {str(e)}")
        
        # エラーメッセージから判断を試みる
        try:
            if 'response_content' in locals():
                # シンプルなヒューリスティックで判断を試みる
                judgement = "含める" in response_content.lower() or "include" in response_content.lower()
                return FullTextDecision(
                    judgement=judgement,
                    reason="APIレスポンスからの抽出に失敗しました。テキスト全体: " + response_content[:200] + "..."
                )
        except:
            pass
        
        # エラーを再度発生させる
        raise


def process_fulltext_abstracts(
    df: pd.DataFrame,
    system_prompt: str,
    client: OpenAI,  # OpenAI クライアントを使用
    text_column: str = "segments",  # フルテキストが入っている列名
    max_retries: int = 3,
    retry_delay: int = 5,
    model: str = "anthropic/claude-3-7-sonnet",  # OpenRouter のモデル名
    checkpoint_interval: int = 10,  # チェックポイントを保存する間隔（レコード数）
    resume_from: int = 0  # 処理を再開するインデックス
) -> List[FullTextScreeningResult]:
    """
    DataFrameの各行に含まれるフルテキストをClaudeに投げて、
    含める／含めない＆理由をまとめて返す。
    """
    results = []
    total_records = len(df)
    start_time = time.time()
    checkpoint_filename = f"screening_checkpoint_{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
    
    # 進捗表示のためのカウンター
    success_count = 0
    error_count = 0
    
    # 進捗バーの初期化
    print(f"処理開始: 全{total_records}件 ({resume_from}件目から開始)")
    print("-" * 50)
    
    # 結果を保存するための一時的なデータフレーム
    temp_df = df.copy()
    temp_df['claude_decision'] = None
    temp_df['claude_reason'] = None
    temp_df['claude_error'] = None

    for i, (idx, row) in enumerate(df.iloc[resume_from:].iterrows(), start=resume_from):
        text_to_screen = row.get(text_column, "")
        current_result = None
        error_message = None
        
        # 進捗を表示
        elapsed_time = time.time() - start_time
        if i > resume_from:
            records_per_second = (i - resume_from) / elapsed_time if elapsed_time > 0 else 0
            estimated_total_time = total_records / records_per_second if records_per_second > 0 else 0
            estimated_remaining = estimated_total_time - elapsed_time if estimated_total_time > 0 else 0
            
            # 進捗率と残り時間を表示
            progress_percent = (i / total_records) * 100
            print(f"\r処理中: {i}/{total_records} ({progress_percent:.1f}%) | "
                  f"成功: {success_count} | エラー: {error_count} | "
                  f"残り約: {estimated_remaining/60:.1f}分", end="")

        # テキストが欠損しているか非常に短い場合はスキップ
        if pd.isna(text_to_screen) or (isinstance(text_to_screen, str) and len(text_to_screen.strip()) < 10):
            print(f"\n[{i}] テキストが欠損または短すぎるためスキップします")
            current_result = FullTextScreeningResult(
                judgement=None,
                reason="テキストが欠損または短すぎるためスキップ",
                error="テキスト不足"
            )
            results.append(current_result)
            
            # 一時的なデータフレームに結果を保存
            temp_df.at[idx, 'claude_decision'] = current_result.judgement
            temp_df.at[idx, 'claude_reason'] = current_result.reason
            temp_df.at[idx, 'claude_error'] = current_result.error
            
            error_count += 1
            continue

        # テキストが長すぎる場合は警告
        if isinstance(text_to_screen, str) and len(text_to_screen) > 15000:
            print(f"\n[{i}] 警告: テキストが長すぎます ({len(text_to_screen)}文字)")

        for attempt in range(max_retries):
            try:
                print(f"\n[{i}] リクエスト開始: {time.strftime('%H:%M:%S')}")
                # Claudeに問い合わせ
                decision = get_fulltext_completion(
                    system_prompt=system_prompt,
                    prompt=text_to_screen,
                    client=client,
                    model=model
                )
                print(f"[{i}] リクエスト完了: {time.strftime('%H:%M:%S')}")
                
                current_result = FullTextScreeningResult(
                    judgement=decision.judgement,
                    reason=decision.reason
                )
                
                success_count += 1
                break  # 成功したらリトライループを抜ける

            except Exception as e:
                error_message = str(e)
                print(f"\n[{i}] Error (attempt {attempt+1}/{max_retries}): {error_message}")
                if attempt < max_retries - 1:
                    print(f"Retrying in {retry_delay} seconds...")
                    time.sleep(retry_delay)
                else:
                    print("Max retries reached. Moving to next.")
                    error_count += 1

        # current_resultに成功結果が入っていなければ、エラーとして記録
        if not current_result:
            current_result = FullTextScreeningResult(
                judgement=None,
                reason=None,
                error=error_message
            )

        results.append(current_result)
        
        # 一時的なデータフレームに結果を保存
        temp_df.at[idx, 'claude_decision'] = current_result.judgement
        temp_df.at[idx, 'claude_reason'] = current_result.reason
        temp_df.at[idx, 'claude_error'] = current_result.error
        
        # チェックポイントを保存
        if (i + 1) % checkpoint_interval == 0:
            temp_df.to_csv(checkpoint_filename, index=False)
            print(f"\nチェックポイント保存: {checkpoint_filename} ({i+1}/{total_records}件完了)")

    # 最終結果を保存
    print("\n" + "-" * 50)
    elapsed_time = time.time() - start_time
    print(f"処理完了: {total_records}件 (所要時間: {elapsed_time/60:.1f}分)")
    print(f"成功: {success_count} | エラー: {error_count}")
    
    return results


def process_fulltext_screening(
    df: pd.DataFrame,
    system_prompt: str,
    client: OpenAI,  # OpenAI クライアントを使用
    text_column: str = "segments",
    model: str = "anthropic/claude-3-7-sonnet",  # OpenRouter のモデル名
    resume_from: int = 0,  # 処理を再開するインデックス
    checkpoint_interval: int = 10  # チェックポイント保存間隔
) -> pd.DataFrame:
    """
    フルテキストスクリーニングを実行し、結果を元のDataFrameに追加して返す

    Args:
        df: 入力DataFrame
        system_prompt: システムプロンプト
        client: OpenAI APIクライアント
        text_column: テキストが含まれる列名
        model: 使用するモデル名
        resume_from: 処理を再開するインデックス
        checkpoint_interval: チェックポイント保存間隔

    Returns:
        結果を追加したDataFrame
    """
    # 元のdfを変更しないようにコピー
    df = df.copy()
    
    # 欠損値のチェック
    missing_count = df[text_column].isna().sum()
    if missing_count > 0:
        print(f"警告: {text_column}列に{missing_count}件の欠損値があります")
    
    # 空文字列や短いテキストのチェック
    empty_text_count = df[text_column].apply(lambda x: isinstance(x, str) and x.strip() == '').sum()
    short_text_count = df[text_column].apply(lambda x: isinstance(x, str) and len(x.strip()) < 10).sum()
    if empty_text_count > 0 or short_text_count > 0:
        print(f"警告: 空テキスト({empty_text_count}件)または短いテキスト({short_text_count}件)があります")

    # データクリーニングを適用
    print("テキストクリーニングを適用中...")
    df[text_column] = df[text_column].apply(clean_text)
    
    # 各テキストに対してスクリーニングを実行
    results = process_fulltext_abstracts(
        df=df,
        system_prompt=system_prompt,
        client=client,
        text_column=text_column,
        model=model,
        resume_from=resume_from,
        checkpoint_interval=checkpoint_interval
    )

    # 結果をDataFrameに追加
    df['claude_decision'] = [r.judgement for r in results]
    df['claude_reason'] = [r.reason for r in results]
    df['claude_error'] = [r.error for r in results]

    return df


# 列の存在確認と要約統計
def validate_results(df):
    # 1. 列が存在するか確認
    expected_columns = ['folder_name', 'claude_decision', 'claude_reason', 'claude_error']
    missing_columns = [col for col in expected_columns if col not in df.columns]
    
    if missing_columns:
        print(f"警告: 以下の列がデータフレームに存在しません: {missing_columns}")
    else:
        print("✓ すべての予想される列が存在します")
    
    # 2. 列の型と非NULLの数を確認
    if 'claude_decision' in df.columns:
        decision_counts = df['claude_decision'].value_counts(dropna=False)
        print("\n[claude_decision の集計]")
        print(f"True の数: {decision_counts.get(True, 0)}")
        print(f"False の数: {decision_counts.get(False, 0)}")
        print(f"NULL の数: {decision_counts.get(None, 0)}")
        if len(df) > 0:
            print(f"判断の割合: {(decision_counts.get(True, 0) / len(df) * 100):.1f}% が含む")
    
    # 3. reasonの長さの統計
    if 'claude_reason' in df.columns:
        # NULL以外のreasonの文字数
        reason_lengths = df['claude_reason'].dropna().apply(len)
        print("\n[claude_reason の統計]")
        print(f"理由の件数: {len(reason_lengths)} / {len(df)}")
        if not reason_lengths.empty:
            print(f"最短の理由: {reason_lengths.min()} 文字")
            print(f"最長の理由: {reason_lengths.max()} 文字")
            print(f"平均文字数: {reason_lengths.mean():.1f} 文字")
    
    # 4. エラーの確認
    if 'claude_error' in df.columns:
        error_count = df['claude_error'].notna().sum()
        print(f"\n[エラー]")
        print(f"エラーの件数: {error_count} / {len(df)}")
        
        if error_count > 0:
            print("最初のエラー例:")
            first_error = df[df['claude_error'].notna()].iloc[0]
            print(f"  フォルダ名: {first_error['folder_name']}")
            print(f"  エラー: {first_error['claude_error'][:100]}..." if len(str(first_error['claude_error'])) > 100 else first_error['claude_error'])
    
    # 5. 処理状況の概要
    total = len(df)
    processed = df['claude_decision'].notna().sum()
    print(f"\n[処理状況]")
    print(f"処理完了: {processed} / {total} ({processed/total*100:.1f}%)")
    
    return None  # 結果の内容は表示せず、統計情報のみ返す



In [ ]:
# 使用例（実行部分）
if __name__ == "__main__":
    # APIキーを設定
    OPENROUTER_API_KEY = " "  # 実際のキーに置き換えてください
    
    # OpenAI クライアントを初期化 (OpenRouter 用に設定)
    client = OpenAI(
        base_url="https://openrouter.ai/api/v1",
        api_key=OPENROUTER_API_KEY
    )
    
    # システムプロンプト例
    system_prompt = """
    You are an expert in systematic reviews of clinical prediction models. Your role is to read full-text articles and decide whether each article should be included or excluded based on the criteria below. You must return a structured response containing:

    A boolean "judgement" field:
    - true if the article meets all inclusion criteria
    - false if the article fails any criterion and should be excluded

    A "reason" field:
    - a concise explanation for why you decided to include or exclude the article

    FULL-TEXT SCREENING CRITERIA

    Inclusion Criteria:
    - Study Focus: Diagnostic prediction models for bacteremia
    - Population: Patients with suspected bacteremia who had blood cultures obtained
    - Model Evaluation: Studies reporting model performance using AUC-ROC, C-statistics, or observed-to-expected events ratio (O:E ratio)
    - Study Design: Any study design except those specified in exclusion criteria
    - Language & Status: Any language, any publication status (including unpublished/conference abstracts)

    Exclusion Criteria:
    - Study Design: Two-gate studies (case-control studies where patients are recruited from different populations)
    - Target Condition: Studies predicting bacteremia of specific microorganisms only (e.g., MRSA, ESBL-producing gram-negative bacteria)
    - Model Complexity: Single predictor studies (models using only one variable such as CRP, procalcitonin, or specific interleukins)
    - Target Condition: Studies where the primary target condition is not bacteremia

    Additional Notes:
    - Studies developing or validating multivariable prediction models should be included
    - No restrictions on publication year, geographical location, or clinical setting
    - External validation studies are eligible if they evaluate a prediction model for bacteremia

    IMPORTANT INSTRUCTIONS FOR THE OUTPUT:
    - judgement must be returned strictly as either true or false
    - reason must always be included, referencing specific criteria
    - Never provide extra fields in the final structured response

    Provide your response using the following XML format:
    <judgement>true or false</judgement>
    <reason>Your detailed justification here</reason>
    """
    
    # ここでデータフレームを読み込む（実際のコードに合わせてください）
    # df = pd.read_csv('your_data.csv')  # 実際のデータファイルパスに置き換え
    
    # 再開するインデックスを指定（0から始める場合は0、途中から再開する場合は該当するインデックス）
    resume_from = 0  # 例: 134から再開する場合は134を指定
    
    # フルテキストスクリーニングを実行
    result_df = process_fulltext_screening(
        df=df_cleaned,  # 実際のデータフレームを使用
        system_prompt=system_prompt,
        client=client,
        model="anthropic/claude-3-7-sonnet",  # OpenRouter で使用可能な Claude 3.7 Sonnet
        resume_from=resume_from,
        checkpoint_interval=10  # 10件ごとにチェックポイントを保存
    )
    
    # 結果を確認（直接内容を見ずに統計情報を確認）
    validate_results(result_df)
    
    # 結果を CSV に保存
    timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    result_filename = f'screening_results_{timestamp}.csv'
    result_df.to_csv(result_filename, index=False)
    print(f"\n結果を '{result_filename}' に保存しました")

In [ ]:
# 元のCSVファイルを読み込む
file_path = 'full_text_without_tab_and_fig_output.csv'  # 実際のファイルパスに置き換え
original_df = pd.read_csv(file_path)

# 列名を確認
print("元のCSVファイルの列名:")
print(original_df.columns.tolist())
print(f"行数: {len(original_df)}")

print("\nresult_dfの列名:")
print(result_df.columns.tolist())
print(f"行数: {len(result_df)}")

# マージのために元のデータフレームのキー列を確認
print("\npdf_key列の最初の5件:")
if 'pdf_key' in original_df.columns:
    print(original_df['pdf_key'].head().tolist())
else:
    print("pdf_key列が存在しません")

print("\npdf_filename列の最初の5件:")
if 'pdf_filename' in original_df.columns:
    print(original_df['pdf_filename'].head().tolist())
else:
    print("pdf_filename列が存在しません")

print("\nfolder_name列の最初の5件:")
print(result_df['folder_name'].head().tolist())


In [ ]:

# マージ方法の決定
# folder_nameがpdf_keyと一致するかpdf_filenameと一致するかを確認
match_with_key = False
match_with_filename = False

# サンプルデータでマッチングを試行
sample_folder_names = result_df['folder_name'].head(5).tolist()

if 'pdf_key' in original_df.columns:
    sample_pdf_keys = original_df['pdf_key'].head(20).astype(str).tolist()
    for name in sample_folder_names:
        if name in sample_pdf_keys:
            match_with_key = True
            print(f"folder_name '{name}' は pdf_key と一致します")
            break

if 'pdf_filename' in original_df.columns:
    sample_pdf_filenames = original_df['pdf_filename'].head(20).astype(str).tolist()
    for name in sample_folder_names:
        if name in sample_pdf_filenames:
            match_with_filename = True
            print(f"folder_name '{name}' は pdf_filename と一致します")
            break

# マージのためのキー列を選択
if match_with_key:
    print("\npdf_key列とfolder_name列でマージを実行します")
    merge_key_original = 'pdf_key'
elif match_with_filename:
    print("\npdf_filename列とfolder_name列でマージを実行します")
    merge_key_original = 'pdf_filename'
else:
    print("\n自動マッチングできませんでした。pdf_key列を使用してマージを試みます")
    merge_key_original = 'pdf_key'


In [ ]:

# マージを実行
# まず型を一致させる
original_df[merge_key_original] = original_df[merge_key_original].astype(str)
result_df['folder_name'] = result_df['folder_name'].astype(str)

# マージの実行
merged_df = pd.merge(
    original_df, 
    result_df, 
    left_on=merge_key_original, 
    right_on='folder_name', 
    how='left'
)

# マージ結果の確認
print(f"\nマージ後のデータフレームの行数: {len(merged_df)}")
print(f"列数: {len(merged_df.columns)}")
print(f"claude_decision列のNull値の数: {merged_df['claude_decision'].isna().sum()}")


In [ ]:
# タイムスタンプを生成
timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")

In [ ]:
# CSVとして保存
csv_filename = f'merged_results_{timestamp}.csv'
merged_df.to_csv(csv_filename, index=False)
print(f"\nCSVファイルとして保存しました: {csv_filename}")

In [ ]:
# Excelとして保存
excel_filename = f'merged_results_{timestamp}.xlsx'
merged_df.to_excel(excel_filename, index=False)
print(f"Excelファイルとして保存しました: {excel_filename}")

In [ ]:

# 保存したファイルのフルパスを表示
current_dir = os.getcwd()
print(f"\nCSVファイルのフルパス: {os.path.join(current_dir, csv_filename)}")
print(f"Excelファイルのフルパス: {os.path.join(current_dir, excel_filename)}")

In [ ]:
# result_dfの列名を表示
print("result_dfの列名:")
print(result_df.columns.tolist())

# 列数も表示
print(f"列数: {len(result_df.columns)}")

In [ ]:
# 使用例
if __name__ == "__main__":


    # システムプロンプト
    system_prompt = """
You are an expert in systematic reviews of clinical prediction models. Your role is to read full-text articles and decide whether each article should be included or excluded based on the criteria below. You must return a structured response containing:

A boolean "judgement" field:
- true if the article meets all inclusion criteria
- false if the article fails any criterion and should be excluded

A "reason" field:
- a concise explanation for why you decided to include or exclude the article

FULL-TEXT SCREENING CRITERIA

Inclusion Criteria:
- Study Focus: Diagnostic prediction models for bacteremia
- Population: Patients with suspected bacteremia who had blood cultures obtained
- Model Evaluation: Studies reporting model performance using AUC-ROC, C-statistics, or observed-to-expected events ratio (O:E ratio)
- Study Design: Any study design except those specified in exclusion criteria
- Language & Status: Any language, any publication status (including unpublished/conference abstracts)

Exclusion Criteria:
- Study Design: Two-gate studies (case-control studies where patients are recruited from different populations)
- Target Condition: Studies predicting bacteremia of specific microorganisms only (e.g., MRSA, ESBL-producing gram-negative bacteria)
- Model Complexity: Single predictor studies (models using only one variable such as CRP, procalcitonin, or specific interleukins)
- Target Condition: Studies where the primary target condition is not bacteremia

Additional Notes:
- Studies developing or validating multivariable prediction models should be included
- No restrictions on publication year, geographical location, or clinical setting
- External validation studies are eligible if they evaluate a prediction model for bacteremia

IMPORTANT INSTRUCTIONS FOR THE OUTPUT:
- judgement must be returned strictly as either true or false
- reason must always be included, referencing specific criteria
- Never provide extra fields in the final structured response

Provide your response using the following XML format:
<judgement>true or false</judgement>
<reason>Your detailed justification here</reason>
    """

    # 処理実行
    df = process_fulltext_screening(
        df=df,
        system_prompt=system_prompt,
        client=client,
        text_column="segments",
        model="gpt-4o"
    )

    # 結果を保存
    df.to_csv("fulltext_screened_results_gpt.csv", index=False)

In [ ]:
df.head()

In [ ]:
#segments列を削除
df_answer = df.drop('segments', axis=1)

In [ ]:
df_answer.to_excel("fulltext_screened_results_gpt.xlsx", index=False)

In [ ]:
#文献データを格納した、スクリーニング用のスプレッドシートを開く

import pandas as pd
import re
from google.colab import auth
from googleapiclient.discovery import build

# Google Colabで認証
auth.authenticate_user()

# スプレッドシートをインポート（自分のスプレッドシートIDに置き換える）
SPREADSHEET_ID = 'your_spreadsheet_id_here'  # スプレッドシートIDを指定
sheet_name =   " "  # シート名を指定

# Google Sheetsから直接データを読み込む
from google.colab import auth
from googleapiclient.discovery import build
import gspread
from google.auth import default
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

# スプレッドシートを開く
worksheet = gc.open_by_key(SPREADSHEET_ID).worksheet(sheet_name)
data = worksheet.get_all_values()
headers = data[0]
df = pd.DataFrame(data[1:], columns=headers)

In [ ]:
pip install rispy

In [ ]:
pip install gdown requests

In [ ]:
import os
import gdown
import re

# ダウンロード先のフォルダを作成
output_dir = "downloaded_files"
os.makedirs(output_dir, exist_ok=True)

# Google DriveリンクのリストからIDを抽出する関数
def extract_file_id(url):
    # Google DriveリンクからIDを抽出
    if 'drive.google.com' in url:
        match = re.search(r'/d/([a-zA-Z0-9_-]+)', url)
        if match:
            return match.group(1)
    # PMCリンクの場合はそのままURLを返す
    elif 'pmc.ncbi.nlm.nih.gov' in url:
        return url
    return None

# リンクのリスト（テキストファイルから読み込むこともできます）
urls = ""

# 改行で分割してリストに変換
url_list = [url.strip() for url in urls.strip().split("\n") if url.strip()]

# 各URLを処理
for i, url in enumerate(url_list):
    file_id = extract_file_id(url)
    
    if file_id:
        if 'pmc.ncbi.nlm.nih.gov' in url:
            # PMCリンクの場合は標準的なダウンロード方法を使用
            import requests
            try:
                response = requests.get(url)
                if response.status_code == 200:
                    filename = os.path.join(output_dir, f"pmc_file_{i+1}.pdf")
                    with open(filename, 'wb') as f:
                        f.write(response.content)
                    print(f"ダウンロード成功: {filename}")
                else:
                    print(f"ダウンロード失敗: {url}")
            except Exception as e:
                print(f"エラー: {url} - {str(e)}")
        else:
            # Google Driveの場合はgdownを使用
            try:
                output_file = os.path.join(output_dir, f"file_{i+1}.pdf")
                gdown.download(id=file_id, output=output_file, quiet=False)
                print(f"ダウンロード成功: {output_file}")
            except Exception as e:
                print(f"エラー: {url} - {str(e)}")
    else:
        print(f"無効なURL: {url}")

print("ダウンロード完了！")